In [ ]:
# -*- coding: UTF-8 -*-
import torch as tc
import cv2
import numpy as np
from tqdm import tqdm

import exp_utils
exp_utils.setup_import_path()
import block

def resize_guiyihua_transpose(img, size=None):
    '''H*W*C*uint8(0\~255) -> C*height*width*float32(-1\~1)\n
    size=None -> height=img.shape[0], width=img.shape[1]\n
    size=integer -> height=width=integer\n
    size=(integer_a, integer_b) -> height=integer_a, width=integer_b'''
    if size is not None:
        if isinstance(size, int):
            height = width = size
        else:
            height, width = size
        img = cv2.resize(img, (width, height),
            interpolation=cv2.INTER_AREA) # height*width*C*uint8
    img = img.astype(np.float32)  # height*width*C*float32
    img = (img - 128) / 128 # norm to -1 ~ 1
    img = np.transpose(img, [2, 0, 1]) # HWC to CHW
    return img

def calcu_mean_and_std(dataset):
    '''Compute the mean and std value of dataset.'''
    dataloader = tc.utils.data.DataLoader(
        dataset, batch_size=1, shuffle=True, num_workers=2)
    mean = tc.zeros(3)
    std = tc.zeros(3)
    print(f'==> Computing mean and std...')
    for input, target in tqdm(dataloader):
        input = resize_guiyihua_transpose(input[0].numpy(), 112)
        input = tc.from_numpy(input)
        for i in range(3):
            mean[i] += input[ i, :, :].mean()
            std[i] += input[ i, :, :].std()
    mean.div_(len(dataset))
    std.div_(len(dataset))
    return mean, std


def get_mean_std(dataset_name):
    ''' according input compute mean and std '''
    if dataset_name == 'celeba':
        print('==>Load celeba......')
        dataset = block.dataset.hubble.xnn_paper.celeba()
        mean, std = calcu_mean_and_std(dataset)
        print('celeba mean and std: ', mean, std)
    elif dataset_name == 'webface':
        print('==>Load webface......')
        dataset = block.dataset.hubble.xnn_paper.webface()
        mean, std = calcu_mean_and_std(dataset)
        print('webface mean and std: ', mean, std)
    elif dataset_name == 'msra':
        print('==>Load msra......')
        dataset = block.dataset.hubble.xnn_paper.msra()
        mean, std = calcu_mean_and_std(dataset)
        print('msra mean and std: ', mean, std)
    else:
        raise ValueError('dataset_name must be celeba, webface or msra')

dataset_names = ['celeba', 'webface', 'msra']
for dataset_name in dataset_names:
    get_mean_std(dataset_name)

In [10]:
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

# Writer will output to ./runs/ directory by default
writer = SummaryWriter(log_dir='/data/privdl_data/log/tmp_utils_ipynb1')

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('/data/privdl_data/datasets/mnist', train=True, download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True)
model = torchvision.models.resnet50(False)
# Have ResNet model take in grayscale rather than RGB
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# images, labels = next(iter(trainloader))

for batch, data in enumerate(trainloader):
    images, labels = data
    grid = torchvision.utils.make_grid(images, nrow=10)
    writer.add_image('images', grid, batch)
    writer.add_graph(model, images)
    if batch == 9:
        break
writer.close()

/home/liukaixin/anaconda3/envs/torch/lib/python3.9/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and will be removed in 0.15. Please use keyword parameter(s) instead.
  warnings.warn(
/home/liukaixin/anaconda3/envs/torch/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [20]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np

tb_log_dir = '/data/privdl_data/log/tmp_utils_ipynb/' + '3_block'

writer = SummaryWriter(log_dir=tb_log_dir)

for n_iter in range(100):
    writer.add_scalar('Loss/train', np.random.random(), n_iter)
    writer.add_scalar('Loss/test', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test', np.random.random(), n_iter)